# Tutorial: Learn machine learning models

This tutorial demonstrates how to learn machine learning models to value on-the-ball actions of football players with the open-source [VAEP framework](https://github.com/ML-KULeuven/socceraction) using the publicly available [Wyscout match event dataset](https://figshare.com/collections/Soccer_match_event_dataset/4415000). The Wyscout dataset includes data for the 2017/2018 English Premier League, the 2017/2018 Spanish Primera División, the 2017/2018 German 1. Bundesliga, the 2017/2018 Italian Serie A, the 2017/2018 French Ligue 1, the 2018 FIFA World Cup, and the UEFA Euro 2016. Covering 1,941 matches, 3,251,294 events and 4,299 players, the dataset is large enough to train machine-learning models and obtain robust ratings for the players.

This tutorial demonstrates the following four steps:
1. Split the dataset into a training set and a test set.
2. Construct the baseline classifiers by using conservative hyperparameters for the learning algorithm.
3. Optimize the classifiers by tuning the hyperparameters for the learning algorithm.
4. Construct the final classifiers using the optimal hyperparameters for the learning algorithm.

This notebook is compatible with `socceraction` version `0.2.0`.

**Conventions:**
* Variables that refer a `DataFrame` object are prefixed with `df_`.
* Variables that refer a collection of `DataFrame` objects (e.g., a list, a set or a dict) are prefixed with `dfs_`.

**References:**
* Tom Decroos, Lotte Bransen, Jan Van Haaren, and Jesse Davis. "[Actions Speak Louder than Goals: Valuing Player Actions in Soccer.](https://arxiv.org/abs/1802.07127)" In *Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining*, pp. 1851-1861. 2019.
* Luca Pappalardo, Paolo Cintia, Alessio Rossi, Emanuele Massucco, Paolo Ferragina, Dino Pedreschi, and Fosca Giannotti. "[A Public Data Set of Spatio-Temporal Match Events in Soccer Competitions.](https://www.nature.com/articles/s41597-019-0247-7)" *Scientific Data 6*, no. 1 (2019): 1-15.

**Optional:** If you run this notebook on Google Colab, then uncomment the code in the following cell and execute the cell.

In [2]:
# !pip install tables==3.6.1
# !pip install scikit-plot

**Optional:** If you run this notebook on Google Colab and wish to store all data in a Google Drive folder, then uncomment the code in the following cell and execute the cell.

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %mkdir -p '/content/gdrive/My Drive/Friends of Tracking/'
# %cd '/content/gdrive/My Drive/Friends of Tracking/'

In [4]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import brier_score_loss, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

# Helper function to plot calibration curve
def plot_calibration_curve(y_true, probas_list, clf_names, n_bins=10, ax=None):
    """Plot calibration curve using sklearn's calibration_curve"""
    if ax is None:
        ax = plt.gca()
    
    for probas, name in zip(probas_list, clf_names):
        fraction_of_positives, mean_predicted_value = calibration_curve(
            y_true, probas, n_bins=n_bins, strategy='uniform'
        )
        ax.plot(mean_predicted_value, fraction_of_positives, 's-', label=name)
    
    ax.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
    ax.set_xlabel('Mean predicted probability')
    ax.set_ylabel('Fraction of positives')
    ax.set_ylim([-0.05, 1.05])
    ax.legend(loc='lower right')
    ax.set_title('Calibration plots')

In [5]:
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [6]:
%matplotlib inline

# Load dataset

This third tutorial assumes that the `spadl.h5` HDF5 file as well as the `features.h5` and `labels.h5` files have been created for a set of games in the first or second tutorial.

This third tutorial only uses features that have been generated in the first tutorial. However, you are strongly encouraged to toy around with the additional features from the second tutorial and to try out your own features to improve the accuracy of the predictive machine learning models!

## Load games

In [7]:
df_games = pd.read_hdf('spadl.h5', key='games')

**Optional:** If you plan to produce results for a particular season in the Wyscout match event dataset (e.g., 2017/2018 English Premier League), you should consider leaving that season out of the dataset to avoid the same game states from appearing in both the training set and the test set. If you would like to only include a particular subset of games, then uncomment the code in the following cell, adapt the selector and execute the cell. The example selector will select all games that were played in the 2017/2018 English Premier League.

In [8]:
# df_games = df_games[
#     df_games['competition_id'] == 364
# ]

In [9]:
df_games.tail(10)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId,groupName,game_id,home_team_id
54,Played,4165363,1,"{'3148': {'scoreET': 0, 'coachId': 134365, 'si...",10078,2018-06-17 15:00:00,15473,Stadion Luzhniki,2057984,"Germany - Mexico, 0 - 1",2018-06-17 17:00:00-02:00,"[{'refereeId': 380599, 'role': 'referee'}, {'r...",Regular,28,Group F,2057984,0
55,Played,4165363,1,"{'16871': {'scoreET': 0, 'coachId': 484354, 's...",10078,2018-06-17 12:00:00,17322,Samara Arena,2057979,"Costa Rica - Serbia, 0 - 1",2018-06-17 14:00:00-02:00,"[{'refereeId': 407722, 'role': 'referee'}, {'r...",Regular,28,Group E,2057979,0
56,Played,4165363,1,"{'16823': {'scoreET': 0, 'coachId': 142913, 's...",10078,2018-06-16 19:00:00,9598,Kaliningrad Stadium,2057973,"Croatia - Nigeria, 2 - 0",2018-06-16 21:00:00-02:00,"[{'refereeId': 378204, 'role': 'referee'}, {'r...",Regular,28,Group D,2057973,0
57,Played,4165363,1,"{'7712': {'scoreET': 0, 'coachId': 54577, 'sid...",10078,2018-06-16 16:00:00,7712,Mordovia Arena,2057967,"Peru - Denmark, 0 - 1",2018-06-16 18:00:00-02:00,"[{'refereeId': 407901, 'role': 'referee'}, {'r...",Regular,28,Group C,2057967,0
58,Played,4165363,1,"{'7839': {'scoreET': 0, 'coachId': 57369, 'sid...",10078,2018-06-16 13:00:00,0,Otkrytiye Arena,2057972,"Argentina - Iceland, 1 - 1",2018-06-16 15:00:00-02:00,"[{'refereeId': 384987, 'role': 'referee'}, {'r...",Regular,28,Group D,2057972,0
59,Played,4165363,1,"{'8493': {'scoreET': 0, 'coachId': 134090, 'si...",10078,2018-06-16 10:00:00,4418,Kazan' Arena,2057966,"France - Australia, 2 - 1",2018-06-16 12:00:00-02:00,"[{'refereeId': 378232, 'role': 'referee'}, {'r...",Regular,28,Group C,2057966,0
60,Played,4165363,1,"{'9905': {'scoreET': 0, 'coachId': 137521, 'si...",10078,2018-06-15 18:00:00,0,Olimpiyskiy Stadion Fisht,2057960,"Portugal - Spain, 3 - 3",2018-06-15 20:00:00-02:00,"[{'refereeId': 377206, 'role': 'referee'}, {'r...",Regular,28,Group B,2057960,0
61,Played,4165363,1,"{'10840': {'scoreET': 0, 'coachId': 136612, 's...",10078,2018-06-15 15:00:00,10840,Stadion Krestovskyi,2057961,"Morocco - Iran, 0 - 1",2018-06-15 17:00:00-02:00,"[{'refereeId': 384995, 'role': 'referee'}, {'r...",Regular,28,Group B,2057961,0
62,Played,4165363,1,"{'16129': {'scoreET': 0, 'coachId': 33204, 'si...",10078,2018-06-15 12:00:00,15670,Stadion Central'nyj,2057955,"Egypt - Uruguay, 0 - 1",2018-06-15 14:00:00-02:00,"[{'refereeId': 377215, 'role': 'referee'}, {'r...",Regular,28,Group A,2057955,0
63,Played,4165363,1,"{'14358': {'scoreET': 0, 'coachId': 264893, 's...",10078,2018-06-14 15:00:00,14358,Olimpiyskiy stadion Luzhniki,2057954,"Russia - Saudi Arabia, 5 - 0",2018-06-14 17:00:00-02:00,"[{'refereeId': 378051, 'role': 'referee'}, {'r...",Regular,28,Group A,2057954,0


## Load features

Load the *features* for the selected games and combine them into the `df_features` `DataFrame` object.

In [10]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    df_features['game_id'] = game_id
    dfs_features.append(df_features)
df_features = pd.concat(dfs_features).reset_index(drop=True)

  0%|          | 0/495 [00:00<?, ?it/s]

In [11]:
df_features.tail(10)

,game_id-0,period_id-0,time_seconds-0,team_id-0,player_id-0,start_x-0,start_y-0,end_x-0,end_y-0,bodypart_id-0,...,end_x_norm-2,end_y_norm-2,end_distance_to_goal-2,end_angle_to_goal-2,xdiff_sequence_pre,ydiff_sequence_pre,time_sequence_pre,xdiff_sequence_post,ydiff_sequence_post,game_id
808548,2057954,1,2863.131899,14358,4513,66,33,53,38,0,...,0.580952,0.176471,49.193496,2.000000,5.0,26.0,2.588698,-8.0,31.0,2057954
808549,2057954,1,2863.807114,16521,122577,47,62,31,36,0,...,0.371429,0.514706,66.007575,66.000000,-2.0,38.0,2.088065,-18.0,12.0,2057954
808550,2057954,1,2865.964868,14358,220971,69,64,73,72,0,...,0.504762,0.558824,52.153619,13.000000,3.0,31.0,2.832969,7.0,39.0,2057954
808551,2057954,1,2868.496551,14358,103682,73,72,78,70,0,...,0.704762,0.470588,31.064449,15.500000,15.0,66.0,4.689437,20.0,64.0,2057954
808552,2057954,1,2870.980334,14358,257800,78,70,77,63,0,...,0.695238,1.058824,49.678969,0.842105,9.0,6.0,5.015466,8.0,-1.0,2057954
808553,2057954,1,2870.980334,16521,122577,22,30,23,37,0,...,0.257143,-0.029412,85.906926,2.166667,-10.0,34.0,2.483783,-9.0,41.0,2057954
808554,2057954,1,2872.843120,16521,122577,23,37,22,30,0,...,0.266667,0.073529,82.280010,2.655172,-4.0,39.0,1.862786,-5.0,32.0,2057954
808555,2057954,1,2939.438099,14358,257800,81,67,0,0,0,...,0.780952,0.455882,23.194827,7.666667,-2.0,29.0,68.457765,-83.0,-38.0,2057954
808556,2057954,1,2943.061365,16521,122561,100,100,19,33,0,...,0.209524,0.441176,83.096330,20.750000,77.0,63.0,70.218245,-4.0,-4.0,2057954
808557,2057954,1,2983.382036,16521,122623,50,50,100,100,0,...,1.000000,1.000000,34.000000,0.000000,26.0,49.0,43.943937,76.0,99.0,2057954


## Load labels

Load the *labels* for the selected games and combine them into the `df_labels` `DataFrame` object.

In [12]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']  
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    df_labels['game_id'] = game_id
    dfs_labels.append(df_labels)
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

  0%|          | 0/495 [00:00<?, ?it/s]

In [13]:
np.sum(df_labels.tail==True)

np.int64(0)

# Split dataset

To accurately predict the labels for game states, the challenge is to train a robust machine learning model that generalizes well from *observed* game states (e.g., game states in games that have been played already) to *unobserved* game states (e.g., game states in games that are yet to be played). Hence, the machine learning model needs to capture interactions between features of game states that are not too specific to identify *similar* game states yet specific enough to distinguish between *dissimilar* game states.

Typically, a machine learning model is said to be *overfitted* on the data if the interactions are too specific (i.e., apply to just a few game states) and is said to be *underfitted* on the data if the interactions are too general (i.e., apply to about all game states). To avoid underfitting, we define an expressive set of features and use a machine learning algorithm that learns a model that can capture complex relationships between these features. To avoid overfitting, we apply [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html) to assess how well a candidate model would perform in practice and apply regularization techniques to keep the learned model as simple as possible.

In this tutorial, we will apply the following methodology:
1. Split the available data into a training set and a test set. The training set will be used to learn the machine learning model, whereas the test set will be kept aside until the very end to assess the real-world performance of the learned model.
2. Learn models with different values for the hyperparameters of the learning algorithm on the training set by adopting a $k$-fold cross-validation setup:
  * Split the training set in $k$ random folds or subsets of equal size.
  * Use $k$-1 training folds to train the model and compute the evaluation metric on the remaining validation fold.
  * Repeat $k$ times until each fold has served as the validation fold once.
3. Select the values for the hyperparameters that yield the best performance across the $k$ validation folds according to the evaluation metric.
4. Train the final model on the full training set using the optimal values for the hyperparameters.
5. Apply the final model on the test set and compute the evaluation metric.

The following cell splits the available data into a training set and test set using the `train_test_split` function.

* **Test set size:** The `test_size` parameter controls the number of examples in the test set. The challenge is to find an appropriate balance between the number of training examples and the number of test examples. Typically, more training examples yield better models, whereas more test examples yield more reliable evaluation metrics.
* **Random state:** The `random_state` parameter sets the *seed* for the random number generator. By setting a *seed*, the train-test split will be the same for each execution of the `train_test_split` function, which is important for reproducing the results and comparing different models.
* **Stratification:** The `stratify` parameter enforces a *stratified* train-test split according to the provided class label. By doing so, the distribution of the provided class label will be the same in the training set and the test set, which can be helpful to obtain a well-calibrated model. Since we will be using two different class labels (i.e., `scores` and `concedes`), we use a concatenation of both class labels for the stratification procedure.

In [14]:
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(
    df_features,
    df_labels,
    test_size=0.10,
    random_state=42,
    stratify=df_labels['scores'].astype(str) + '_' + df_labels['concedes'].astype(str)
)

The following cells inspect whether the training set and test set have the same proportion of positive and negative examples for each class.

In [15]:
df_y_train['scores'].mean()

np.float64(0.0)

In [16]:
df_y_test['scores'].mean()

np.float64(0.0)

In [17]:
df_y_train['concedes'].mean()

np.float64(0.0)

In [18]:
df_y_test['concedes'].mean()

np.float64(0.0)

**Note:** In a real-world scenario where more data is available, you should consider respecting the chronological order of the games to construct the training set, validation set and test set. For instance, use the data for the 2016/2017 and 2017/2018 seasons to train the models, use the data for the 2018/2019 season to tune the models, and use the data for the 2019/2020 season to obtain the results.

# Construct baseline classifiers

The following cell provides a list of features that the machine learning algorithm will consider to train the model.

In [19]:
features = [
    'start_distance_to_goal-0',
    'end_distance_to_goal-0',
    'start_distance_to_goal-1',
    'end_distance_to_goal-1',
    'start_distance_to_goal-2',
    'end_distance_to_goal-2',
    'start_angle_to_goal-0',
    'end_angle_to_goal-0',
    'start_angle_to_goal-1',
    'end_angle_to_goal-1',
    'start_angle_to_goal-2',
    'end_angle_to_goal-2',
    'team-1',
    'team-2'
]

The following cell provides a list of class labels for which the machine learning algorithm will train a model.

**Note:** The `concedes` class label has been commented to speed up the execution of the entire notebook.

In [20]:
labels = [
    'scores',
#     'concedes'
]

## Train classifiers

The following cell trains an XGBoost classifier for each label using *conservative* hyperparamters for the learning algorithm, which will serve as *baseline* models.

In [21]:
%%time
models = {}
for label in tqdm(labels):
    model = XGBClassifier(
        eval_metric='logloss',
        use_label_encoder=False,
        n_estimators=10,
        max_depth=3,
        base_score=0.5
    )
    model.fit(
        X=df_X_train[features],
        y=df_y_train[label]
    )
    models[label] = model

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\anma10\PycharmProjects\fot-valuing-actions\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:54:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


CPU times: total: 17.4 s
Wall time: 718 ms


## Estimate probabilities

The following cell estimates the probabilities for each label using the trained *baseline* models.

In [22]:
dfs_predictions = {}
for label in tqdm(labels):
    model = models[label]
    probabilities = model.predict_proba(
        df_X_test[features]
    )
    predictions = probabilities[:, 1]
    dfs_predictions[label] = pd.Series(predictions, index=df_X_test.index)
df_predictions = pd.concat(dfs_predictions, axis=1)

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
df_predictions


,scores
199132,0.022262
216937,0.022262
26770,0.022262
666573,0.022262
630527,0.022262
...,...
50082,0.022262
550648,0.022262
552616,0.022262
93778,0.022262


In [24]:
# Check if features exist
print("Features in training set:", features)
print("Missing features:", [f for f in features if f not in df_X_train.columns])
print("\nFeature stats:")
print(df_X_train[features].describe())

# Check label distribution
print("\nLabel distribution:")
print(df_y_train['scores'].value_counts())
print("Positive rate:", df_y_train['scores'].mean())

# Check training accuracy
for label in labels:
    model = models[label]
    train_preds = model.predict(df_X_train[features])
    train_acc = (train_preds == df_y_train[label].values).mean()
    print(f"{label} training accuracy: {train_acc:.4f}")

Features in training set: ['start_distance_to_goal-0', 'end_distance_to_goal-0', 'start_distance_to_goal-1', 'end_distance_to_goal-1', 'start_distance_to_goal-2', 'end_distance_to_goal-2', 'start_angle_to_goal-0', 'end_angle_to_goal-0', 'start_angle_to_goal-1', 'end_angle_to_goal-1', 'start_angle_to_goal-2', 'end_angle_to_goal-2', 'team-1', 'team-2']
Missing features: []

Feature stats:
       start_distance_to_goal-0  end_distance_to_goal-0  \
count             727702.000000           727702.000000   
mean                  66.764707               64.602527   
std                   22.378271               23.133267   
min                    5.000000                5.000000   
25%                   50.447993               47.507894   
50%                   67.742158               66.128662   
75%                   82.800966               80.056230   
max                  124.020160              124.020160   

       start_distance_to_goal-1  end_distance_to_goal-1  \
count             7

In [25]:
df_y_train['scores'].value_counts()
df_y_test['scores'].value_counts()

scores
False    80856
Name: count, dtype: int64

## Evaluate probabilities

### Compute base rate probabilities

The following cell computes the *base rate* or *prior probability* of each class label in the training set. We use the *base rate* as a naive estimate for each example in the test set being true to establish a baseline for the evaluation metrics.

In [26]:
df_base_rates = pd.DataFrame({
    label:np.full(len(df_y_test[label]), df_y_train[label].mean()) for label in labels
})

In [27]:
df_base_rates.head(10)

,scores
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


### Compute Brier score loss for goal scored model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [28]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_base_rates['scores']
)

TypeError: missing a required argument: 'y_proba'

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_predictions['scores']
)

### Compute Brier score loss for goal conceded model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_base_rates['concedes']
# )

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_predictions['concedes']
# )

### Plot calibration curve and probability histogram

The following cell creates a plot to show both a calibration curve and a probability histogram.

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(8, 8),
    gridspec_kw={
        'height_ratios': [3, 1]
    }
)

The following cell plots the *calibration curve* for the trained model.

In [ ]:
plot_calibration_curve(
    y_true=df_y_test['scores'],
    probas_list=[df_predictions['scores'].tolist()],
    clf_names=['Goal scored model'],
    n_bins=10,
    ax=ax1
)

The following cell plots the histogram of the predictions produced by the trained model. Clearly, the majority of the examples is *negative*, which means that the start of the calibration curve is extremely important.

In [ ]:
df_predictions['scores'].plot.hist(
    range=(0, 1),
    bins=10,
    ax=ax2
)

# Optimize classifiers

In order to train *accurate* models, we perform a *grid search* over different combinations of parameter values for the most important hyperparameters for the learning algorithm. We will focus on the number of estimators and the maximum depth of the decision trees although more hyperparameters influence the performance of the trained models. The more combinations of parameter values need to be explored, the longer the *grid search* will take.

Furthermore, restricting the number of estimators (i.e., the number of decision trees) and the maximum depth of the decision trees is an important mechanism to reduce the complexity of the trained models and thus also to avoid overfitting on the training data. The more decision trees that the learning algorithm can use and the deeper these decision trees can become, the more opportunities the learning algorithm has to *memorize* the training data rather than to discover patterns that generalize to the unseen test data.

## Train classifiers

The following cell trains an XGBoost classifier for each label by trying different combinations of hyperparameters.

In [ ]:
%%time
models_cv = {}
for label in tqdm(labels):
    model = GridSearchCV(
        estimator=XGBClassifier(
            eval_metric='logloss',
            use_label_encoder=False,
        ),
        param_grid={
            'n_estimators': [50, 100],
            'max_depth': [3, 4]
        },
        scoring='neg_brier_score',
        refit=True,  # train final model on full training set using best hyperparameters
        verbose=10,
        n_jobs=1
    )
    model.fit(
        X=df_X_train[features],
        y=df_y_train[label]
    )
    models_cv[label] = model

CPU times: total: 0 ns
Wall time: 0 ns


NameError: name 'tqdm' is not defined

**Note:** We have considered a manually selected set of features to represent the game states. In addition to optimizing the hyperparameters for the learning algorithm, we could also optimize the set of features to be considered by the learning algorithm. However, the XGBoost algorithm should be able to figure out by itself which features are most important to include in the model by the nature of the algorithm.

## Estimate probabilities

The following cell estimates the probabilities for each label using the trained *baseline* models.

In [ ]:
dfs_predictions_cv = {}
for label in tqdm(labels):
    model = models_cv[label]
    probabilities = model.predict_proba(
        df_X_test[features]
    )
    predictions = probabilities[:, 1]
    dfs_predictions_cv[label] = pd.Series(predictions, index=df_X_test.index)
df_predictions_cv = pd.concat(dfs_predictions, axis=1)

## Evaluate probabilities

### Compute Brier score loss for goal scored model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_base_rates['scores']
)

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_predictions_cv['scores']
)

### Compute Brier score loss for goal conceded model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_base_rates['concedes']
# )

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_predictions_cv['concedes']
# )

### Plot calibration curve and probability histogram

The following cell creates a plot to show both a calibration curve and a probability histogram.

In [ ]:
fig_cv, (ax1_cv, ax2_cv) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(8, 8),
    gridspec_kw={
        'height_ratios': [3, 1]
    }
)

The following cell plots the *calibration curve* for the trained model.

In [ ]:
plot_calibration_curve(
    y_true=df_y_test['scores'],
    probas_list=[df_predictions_cv['scores'].tolist()],
    clf_names=['Goal scored model'],
    n_bins=10,
    ax=ax1_cv
)

The following cell plots the histogram of the predictions produced by the trained model. Clearly, the majority of the examples is *negative*, which means that the start of the calibration curve is extremely important.

In [ ]:
df_predictions_cv['scores'].plot.hist(
    range=(0, 1),
    bins=10,
    ax=ax2_cv
)

## Optional: Calibrate probabilities

If the trained model produces poorly calibrated probability estimates, the probability estimates can be re-calibrated using [CalibratedClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html), which performs probability calibration with isotonic regression or logistic regression.

# Construct final classifiers

Once we have found the *best* feature set and *best* hyperparameters for the learning algorithm, we can learn the final model.

1. If we use `GridSearchCV` and the `refit` parameter was set to `True`, we can retrieve the *final* model, which has been re-trained on the entire training set, by accessing the `best_estimator_` attribute of the object.
2. We can manually train the *final* model by creating a `XGBClassifier` object using the *best* hyperparameters and calling the `fit` method with the entire training set.

## Option 1: Retrieve classifier from grid search

In [ ]:
model_scores = models_cv['scores']

The following cell shows the *best* hyperparameter combination that was found using the *grid search*.

In [ ]:
pd.Series(
    model_scores.best_params_
)

The following cell shows the full results of the *grid search*.

In [ ]:
pd.DataFrame(
    model_scores.cv_results_
)

The following cell shows the hyperparameters that were used to re-train the model on the entire training set.

In [ ]:
pd.Series(
    model_scores.best_estimator_.get_params()
)

The following cell retrieves the final `XGBClassifier` object from the `GridSearchCV` object.

In [ ]:
model_scores_final = model_scores.best_estimator_

In [ ]:
model_scores_final

## Option 2: Train classifier using optimal hyperparameters

The following cell constructs a new `XGBoostClassifier` object using the *best* hyperparameters that were found by the *grid search*.

In [ ]:
model_scores_final = XGBClassifier(
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=4
)

The following cell fits the `XGBoostClassifier` object on the entire training set.

In [ ]:
model_scores_final.fit(
    X=df_X_train[features],
    y=df_y_train[label]
)